<a href="https://colab.research.google.com/github/RoomRooms/2023BigData/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
cd drive/MyDrive/BigData/csv

In [ ]:
!pip install pytorch_tabnet

In [ ]:
# 데이터를 저장할 빈 데이터프레임 생성
all_data = pd.DataFrame()
filename = ['10-Rbot-20110818.binetflow.parquet', '4-Rbot-20110815.binetflow.parquet', '11-Rbot-20110818-2.binetflow.parquet', '5-Virut-20110815-2.binetflow.parquet', '12-NsisAy-20110819.binetflow.parquet', '6-Menti-20110816.binetflow.parquet', '13-Virut-20110815-3.binetflow.parquet', '7-Sogou-20110816-2.binetflow.parquet', '1-Neris-20110810.binetflow.parquet', '8-Murlo-20110816-3.binetflow.parquet', '2-Neris-20110811.binetflow.parquet', '9-Neris-20110817.binetflow.parquet', '3-Rbot-20110812.binetflow.parquet']

# 13개의 parquet 파일 순회
for file_path in filename:
    # parquet 파일 로드
    df = pd.read_parquet(file_path)
    
    # 데이터 프레임 연결
    all_data = pd.concat([all_data, df], ignore_index=True)

In [ ]:
# NaN Drop
all_data = all_data.dropna()
all_data.reset_index(inplace=True, drop=True)

In [ ]:
label_encoder = LabelEncoder()
tmp = pd.DataFrame()
tmp['encode'] = label_encoder.fit_transform(all_data['label'])
tmp['label'] = all_data['label']
print(tmp.columns)

In [ ]:
en = []
la = []
for encode, label in tmp.values:
    if('flow=From-Botnet' in label):
        en.append(encode)
        la.append(label)

In [ ]:
tmp2 = pd.DataFrame({'Encode' : en, 'Label':la})

In [ ]:
ls_bot = list(tmp2['Encode'].unique())

In [ ]:
# Label Encoder 생성 및 적용
label_encoder = LabelEncoder()
all_data['proto'] = label_encoder.fit_transform(all_data['proto'])
all_data['dir'] = label_encoder.fit_transform(all_data['dir'])
all_data['state'] = label_encoder.fit_transform(all_data['state'])
all_data['Family'] = label_encoder.fit_transform(all_data['Family'])
all_data['label'] = label_encoder.fit_transform(all_data['label'])

all_data['target'] = None

In [ ]:
count = 0

for label in all_data['label']:
    if label in ls_bot:
        all_data.loc[count, 'target'] = 1
    else:
        all_data.loc[count, 'target'] = 0
    count += 1

In [ ]:
# 입력 데이터와 출력 데이터 분리
X = all_data[['dur', 'proto', 'dir', 'state', 'stos', 'dtos', 'tot_pkts', 'tot_bytes', 'src_bytes', 'Family', 'label']]
Y = all_data['target']

# Label Encoding
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

# 학습 데이터와 테스트 데이터 분리
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=42)

# 검증 데이터 분리
train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size=0.2, random_state=42)

In [ ]:
# TabNet 모델 생성 및 학습
model = TabNetClassifier()
model.fit(train_X.values, train_Y, eval_set=[(val_X.values, val_Y)])

In [ ]:
# 테스트 데이터 예측
preds = model.predict(test_X.values)

In [ ]:
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve, f1_score, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
# 성능 지표 계산
f1 = f1_score(test_Y, preds)
accuracy = accuracy_score(test_Y, preds)

# 분류 보고서 출력
report = classification_report(test_Y, preds)
print(report)

# ROC Curve 계산
fpr, tpr, _ = roc_curve(test_Y, preds)
auc = roc_auc_score(test_Y, preds)

# Precision-Recall Curve 계산
precision, recall, _ = precision_recall_curve(test_Y, preds)

# 성능 지표 출력
print(f"F1-Score: {f1}")
print(f"AUC: {auc}")
print(f"Accuracy: {accuracy}")

# ROC Curve 시각화
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve 시각화
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

In [ ]:
import numpy as np 
import pandas as pd 

fns = []
import os
from google.colab import drive
drive.mount('/content/drive')
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         fns.append(os.path.join(dirname, filename))
# print(fns)

In [ ]:
from scapy.all import rdpcap, Ether, ARP, TCP, UDP, ICMP
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
packets = rdpcap('./1/botnet-capture-20110810-neris.pcap')

In [ ]:
df = pd.read_csv('./1/capture20110810.binetflow')

In [ ]:
print(packets)

In [ ]:
count = 0
for packet in packets:
    if(count>50):
        break
    print(packet)
    count += 1

In [ ]:
count = 0
for packet in packets:
    if(count>5):
        break
    print(packet[IP])
    count += 1

In [ ]:
cd csv

In [ ]:
ls

In [ ]:
for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        fns.append(os.path.join(dirname, filename))
print(fns)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score
from fastcore.basics import *
from fastcore.parallel import *
from os import cpu_count
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
df = pd.concat(objs=[
    pd.read_parquet(fns[0])], copy=False, sort=False)

In [ ]:
df.label.value_counts().head(10)

In [ ]:
df['dir']

In [ ]:
df = pd.concat(objs=[
    pd.read_parquet(f) for f in fns
], copy=False, sort=False)
df.shape, df.columns

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        ###############################
        ########## 문제 3-1 ############

        self.conv1_1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(32)
        self.conv1_2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn1_2 = nn.BatchNorm2d(64)

        self.conv2_1 = nn.Conv2d(64, 94, kernel_size=3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(94)
        self.conv2_2 = nn.Conv2d(94, 128, kernel_size=3, stride=1, padding=1)
        self.bn2_2 = nn.BatchNorm2d(128)

        self.conv3_1 = nn.Conv2d(128, 192, kernel_size=3, stride=1, padding=1)
        self.bn3_1 = nn.BatchNorm2d(192)
        self.conv3_2 = nn.Conv2d(192, 256, kernel_size=3, stride=1, padding=1)
        self.bn3_2 = nn.BatchNorm2d(256)

        self.conv4_1 = nn.Conv2d(256, 320, kernel_size=3, stride=1, padding=1)
        self.bn4_1 = nn.BatchNorm2d(320)
        self.conv4_2 = nn.Conv2d(320, 384, kernel_size=3, stride=1, padding=1)
        self.bn4_2 = nn.BatchNorm2d(384)
        self.conv4_3 = nn.Conv2d(384, 448, kernel_size=3, stride=1, padding=1)
        self.bn4_3 = nn.BatchNorm2d(448)
        self.conv4_4 = nn.Conv2d(448, 512, kernel_size=3, stride=1, padding=1)
        self.bn4_4 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512 * 1 * 1, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 10)

        ###############################
    def forward(self, x):
        x = F.relu(self.bn1_1(self.conv1_1(x)))
        x = F.relu(self.pool(self.bn1_2(self.conv1_2(x))))

        x = F.relu(self.bn2_1(self.conv2_1(x)))
        x = F.relu(self.pool(self.bn2_2(self.conv2_2(x))))

        x = F.relu(self.bn3_1(self.conv3_1(x)))
        x = F.relu(self.pool(self.bn3_2(self.conv3_2(x))))

        x = F.relu(self.bn4_1(self.conv4_1(x)))
        x = F.relu(self.bn4_2(self.conv4_2(x)))
        x = F.relu(self.bn4_3(self.conv4_3(x)))
        x = F.relu(self.pool(self.bn4_4(self.conv4_4(x))))
        
        x = torch.nn.Flatten()(x) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)

        x = self.fc3(x)
        return x

net = CNN()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

In [ ]:
def create_rnn_model(input_shape):
    # 입력 레이어 정의
    inputs = Input(shape=input_shape)

    # RNN 레이어 정의
    rnn_layer = LSTM(64)(inputs)

    # 출력 레이어 정의
    outputs = Dense(1, activation='sigmoid')(rnn_layer)

    # 모델 생성
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
print(df.shape)
print(df.isna().sum(axis=0))

In [ ]:
df['label'] = df['label'].astype(dtype='object')
df['label'] = df['label'].str.startswith('flow=From-Botnet', na=False)
df['label'] = df['label'].astype(dtype='float32', copy=False)
df.label.value_counts().head(10)

In [ ]:
df = df.drop(columns=['Family'])
df.shape

In [ ]:
print(df.shape)
print(df.isna().sum(axis=0))

In [ ]:
df.head

In [ ]:
df['proto'] = df['proto'].astype('category').cat.codes
df['proto'] = df['proto'].astype(np.int32)
df['dir'] = df['dir'].astype('category').cat.codes
df['dir'] = df['dir'].astype(np.int32)
df['state'] = df['state'].astype('category').cat.codes
df['state'] = df['state'].astype(np.int32)

In [ ]:
df.isna().sum(axis=0)

In [ ]:
df_test = df.sample(frac=0.2, replace = False)
df_train = df.drop(index = df_test.index)

In [ ]:
test_Y = df_test['label']
test_X = df_test.drop('label', axis=1)
train_Y = df_train['label']
train_X = df_train.drop('label', axis=1)

In [ ]:
train_X.shape

In [ ]:
# 입력 데이터의 shape 정의
timesteps = 9  # 시간 단계 수
input_dim = 9  # 입력의 차원

# 입력 레이어 정의
inputs = Input(shape=(timesteps, input_dim))

# LSTM 레이어 정의
lstm_units = 32  # LSTM의 출력 차원 수
lstm = LSTM(units=lstm_units)(inputs)

# 출력 레이어 정의
output_units = 1  # 출력의 차원 수
outputs = Dense(units=output_units, activation='sigmoid')(lstm)  # 분류 문제의 경우 units는 클래스 수

# 모델 생성
model = Model(inputs=inputs, outputs=outputs)

# 모델 요약 정보 출력
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 입력 데이터 reshape
train_X_reshaped = np.expand_dims(train_X, axis=2)  # 입력 데이터의 차원을 3D로 변경

# 모델 학습
epochs = 10
batch_size = 32

model.fit(train_X_reshaped, train_Y, epochs=epochs, batch_size=batch_size)

# 예측 데이터 reshape
test_X_reshaped = np.expand_dims(test_X, axis=2)  # 예측 데이터의 차원을 3D로 변경

# 모델 예측
y_pred = model.predict(test_X_reshaped)

In [ ]:
for batch in dataloader:
    print(batch[0])

In [ ]:
df_test = df.sample(frac=0.2, replace = False)
df_train = df.drop(index = df_test.index)
df_train.shape, df_test.shape

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)

net = net.to(device) # device로 Network 전송
print(net) # 모델 구조 확인

In [ ]:
batch_size = 16

In [ ]:
train_loader = torch.utils.data.DataLoader(df_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(df_test, batch_size=batch_size, shuffle=True)

In [ ]:
print(df_test.shape)

In [ ]:
training_epochs = 5
for epoch in range(training_epochs): # training_epochs 수만큼 반복
    avg_loss = 0
    for X_train, target in df_train:
        pred = net(X_train) # 
        optimizer.zero_grad() 
        loss = criterion(pred, target) # Model output과 target 비교
        loss.backward() # gradient 계산
        optimizer.step() # parameter 업데이트
        avg_loss += loss / len(train_loader) 
    print('[Epoch: {:>3}] loss = {:>.9}'.format(epoch + 1, avg_loss))
 

In [ ]:
for X, y in train_loader:
    print(X, y)

In [ ]:
# training
EPOCHS = 20
for epoch in range(EPOCHS): # epochs수만큼 반복
    avg_loss = 0
    for data, target in train_loader:
        # print(data.shape)
        data = data.cuda() # 데이터도 gpu로 보내야함
        target = target.cuda()
        pred = net(data) # 
        # print(pred)
        optimizer.zero_grad() 
        loss = criterion(pred, target) # Model output과 target 비교
        loss.backward() # gradient 계산
        optimizer.step() # parameter 업데이트
        avg_loss += loss / len(train_loader) 
    print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch + 1, avg_loss))

In [ ]:
print(df_train.index)

In [ ]:
print(train_loader.index)